In [1]:
import smtplib
import ssl
import time
import json
import random
from datetime import datetime as dt
from IPython.display import clear_output
from email.message import EmailMessage
from tgtg import TgtgClient

### Reinitiate login

In [2]:
client = TgtgClient(email="jendrik.itzen@gmx.de")
credentials = client.get_credentials()

Using version 23.5.1
Check your mailbox on PC to continue... (Mailbox on mobile won't work, if you have installed tgtg app.)
Check your mailbox on PC to continue... (Mailbox on mobile won't work, if you have installed tgtg app.)
Logged in!


### Set up automation class

In [3]:
class TGTG_Automation:
    def __init__(self, fav_stores_to_check, client, mail_receiver, all_favorites=False):
        self.client = client
        self.mail_receiver = mail_receiver
        self.standard_phrase = 'Restaurants and available bags:\n'
        self.current_text = self.standard_phrase
        self.new_info = False
        self.showed_favorites = False
        self.fav_stores_to_check = fav_stores_to_check  # Stores that should be sent via mail
        self.stores = {}  # Favorites that are ticked in the app, should be sent via mail and have stock
        self.all_favorites = all_favorites

    def get_available_favorites(self, show_current_favorites=False):
        self.stores = {}
        items = self.client.get_items()  # All information about all favourites in app
        if show_current_favorites and not self.showed_favorites:
            print([item['store']['store_name'] for item in items])
            self.showed_favorites = True
        for item in items:
            name = item['store']['store_name']
            if name in self.fav_stores_to_check or self.all_favorites:
                items_available = item['items_available']
                if items_available > 0:
                    self.stores[name] = items_available

    def update_current_text(self):
        text = self.standard_phrase
        for key, val in zip(self.stores.keys(), self.stores.values()):
            key = key.replace('\n', '')
            text += f"\n{key}: {val}"
        if text != self.current_text:
            self.current_text = text
            self.new_info = True
    
    def update_user_agent(self):
        self.client.user_agent = self.client._get_user_agent()
        print(self.client.user_agent)

    def send_mail(self):
        if self.new_info:
            pw = 'hvrhddejlfnvkrqw'
            sender = 'petermeyer123321@gmail.com'
            subject = 'Available bags updated!'
            body = self.current_text

            em = EmailMessage()
            em['From'] = f'TGTG API <{sender}>'
            em['To'] = self.mail_receiver
            em['Subject'] = subject
            em.set_content(body)

            context = ssl.create_default_context()

            with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
                smtp.login(sender, pw)
                smtp.sendmail(sender, self.mail_receiver, em.as_string())
            print(f'Mail sent! ({dt.now()})')
            self.new_info = False
        else:
            print(f'No update! ({dt.now()})')

### Define favorites to check and sent per mail

In [22]:
relevant_favorites = ['Chicago Bagels', 'Roofers Kitchen', 'Café Knoll', "Schäfer's", 'Kamps Bäckerei']

my_client = TgtgClient(access_token=credentials['access_token'],
                refresh_token=credentials['refresh_token'],
                user_id=credentials['user_id'],
                cookie=credentials['cookie'])

refresher = TGTG_Automation(relevant_favorites, my_client, 'jendrik.itzen@gmx.de', True)

while True:
    clear_output()
    try:
        refresher.get_available_favorites()
        refresher.update_current_text()
        refresher.send_mail()
        refresher.update_user_agent()
    except:
        print(f'An error occured at: ({dt.now()})')
        time.sleep(60)
    time.sleep(int(max(random.normalvariate(180, 20), 40)))
    
    

ok
Mail sent! (2023-06-15 15:18:36.526603)
Using version 23.5.1
TGTG/23.5.1 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1


KeyboardInterrupt: 

In [ ]:
# In case the package gets updated, these are the user agents:
USER_AGENTS = [
    'TGTG/{} (Linux; U; Android 9; Nexus 5 Build/M4B30Z)',
    'TGTG/{} (Linux; U; Android 10; Pixel 3 Build/QP1A.190711.020)',
    'TGTG/{} (Linux; U; Android 11; Galaxy S20 Build/RP1A.200720.012)',
    'TGTG/{} (Linux; U; Android 12; OnePlus 8T Build/RD1A.201105.003)',
    'TGTG/{} (Linux; U; Android 9; Moto G7 Build/PDYS29.105-165-10)',
    'TGTG/{} (Linux; U; Android 10; LG V60 ThinQ Build/QKQ1.200216.002)',
    'TGTG/{} (Linux; U; Android 11; Sony Xperia 1 II Build/58.1.A.5.159)',
    'TGTG/{} (Linux; U; Android 12; Xiaomi Mi 11 Build/RKQ1.201112.002)',
    'TGTG/{} (Linux; U; Android 9; Huawei P30 Pro Build/HUAWEIVOG-L29)',
    'TGTG/{} (Linux; U; Android 10; Samsung Galaxy Note10+ Build/QP1A.190711.020)',
    'TGTG/{} (iPhone; CPU iPhone OS 13_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.2 Mobile/15E148 Safari/604.1',
    'TGTG/{} (iPhone; CPU iPhone OS 14_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1',
    'TGTG/{} (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1',
    'TGTG/{} (iPhone; CPU iPhone OS 13_6_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.2 Mobile/15E148 Safari/604.1',
    'TGTG/{} (iPhone; CPU iPhone OS 14_0_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1',
    'TGTG/{} (iPhone; CPU iPhone OS 15_0_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1',
    'TGTG/{} (iPhone; CPU iPhone OS 13_7 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.2 Mobile/15E148 Safari/604.',
    'TGTG/{} (iPhone; CPU iPhone OS 14_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/'
]